In [1]:
using Pkg
Pkg.activate("../.")
using Revise
using MeshCat
using MeshCatMechanisms
using DelimitedFiles
using TOML
using Pkg 
using RigidBodyDynamics 
using Rotations: rotation_error, CayleyMap, UnitQuaternion 
using StaticArrays
using ForwardDiff
using LinearAlgebra
using Rotations
using PyPlot 
using QuadrupedBalance
using SparseArrays
using RobotDynamics
const QB = QuadrupedBalance

WebIO._IJuliaInit()

  Activating project at `~/dev/QuadrupedBalance.jl`


QuadrupedBalance

In [38]:
"""Take a single semi-implicit euler integration step. Returns x_next and λ"""
function semi_implicit_euler(A1::QB.AbstractQuadruped, x::AbstractVector, u::AbstractVector, ϕ_cons::AbstractVector, foot_indices, h)
    xn = copy(x)
    
    # Mass matrix and dynamics bias 
    M = QB.get_mass_matrix(A1, xn)
    C_dyn = QB.get_dynamics_bias(A1, xn)
    
    # Kinematic Jacobian 
    attitude_error_jacobian = blockdiag(sparse(0.5*QuadrupedBalance.quaternion_differential(x[1:4])),
                                        sparse(Rotations.UnitQuaternion(x[1:4])),
                                        sparse(I(30)) )
    J = QB.dfk_world(x)[foot_indices,:] * attitude_error_jacobian
    J = J[:, 1:18]
    ϕ = QB.fk_world(xn)[foot_indices]

    v_dim = num_velocities(A1.rigidbody.mech)
    p_dim = num_positions(A1.rigidbody.mech)
    
    ## Solving for velocities 
    damp = 0.5
    r = [([zeros(6);u].-C_dyn - [zeros(6);damp*xn[p_dim+6+1:end]])*h.+M*xn[p_dim+1:end]; -ϕ+ϕ_cons] # KKT conditions 
    H = zeros(v_dim+6, v_dim+6)  
    H[1:v_dim,1:v_dim] .= M 
    H[1:v_dim,v_dim+1:end] .= J'*h 
    H[v_dim+1:end, 1:v_dim] .= J*h
    δ = H \ r

    v_next = δ[1:v_dim]
    λ = δ[v_dim+1:end]

    ## Integrating 
    rot = UnitQuaternion(xn[1:4])
    v_trans = rot * v_next[4:6]   
    xn[5:7] = xn[5:7] + v_trans * h
    xn[8:p_dim] .= xn[8:p_dim] + v_next[7:end] *h


    # Update attitude
    q_dot = 0.5*QB.get_G(x[1:4])*v_next[1:3]*h
    xn[1:4] .= xn[1:4] + q_dot
    xn[1:4] .= xn[1:4]/norm(x[1:4])
    xn[p_dim+1:end] .= v_next 
    return xn, λ
end 

semi_implicit_euler

In [3]:
#### Loading eq point, and setting which foot are in contacts 
data = TOML.parsefile("ipopt_eq_point.toml") # load eq point 
x_eq = data["x_eq"];
u_eq = data["u_eq"];
λ_eq = data["λ_eq"];

urdfpath = joinpath(@__DIR__, "..", "src","a1","urdf","a1.urdf")
A1mech = parse_urdf(urdfpath, floating=true, remove_fixed_tree_joints=false)
A1 = QuadrupedBalance.UnitreeA1FullBody(A1mech);
foot_contacts = [1, 0, 0, 1] # FR, FL, RR, RL
foot_indices = []            # specifies which foot constraints are active 
for i in 1:length(foot_contacts)
    if(foot_contacts[i] == 1)
        append!(foot_indices, (i-1)*3 .+ (1:3))
    end 
end 


In [50]:
findbody(A1mech, "trunk").spatial_inertia

LoadError: type RigidBody has no field spatial_inertia

In [39]:
###### Simulation Loop ######
n = length(x_eq)
m = 12
h = 0.001
tf = 5.0
times = 0:h:tf
ϕ_cons = QB.fk_world(x_eq)[foot_indices] # feet constraint 

# Load control gains
K = readdlm("maximal_lqr_gain.txt", '\t', Float64, '\n')

# Loop variables 
xs = zeros(length(times), n)
us = zeros(length(times)-1, m)
λs = zeros(length(times), 6)
x_errs = zeros(length(times)-1, n-1)
xs[1,:] = copy(x_eq)

com_offset = [0.0, 0.0, 0.0] # breaks after 0.01 shifts in any direction

for i in 1:length(times) - 1
    ## get sensor 
    encs = xs[i, 8:19]
    joint_vels = xs[i, 26:end]
    pos = xs[i,5:7]
    v = xs[i,23:25]
    ω = xs[i,20:22]
    quat_meas = UnitQuaternion(xs[i,1:4])
    quat_eq = UnitQuaternion(x_eq[1:4])
    
    ## Feedback control 
    x_errs[i, 4:6] =  pos[1:3] - x_eq[5:7] - (quat_meas' * com_offset)
    θ_err = rotation_error(quat_meas, quat_eq, Rotations.CayleyMap()) 
    x_errs[i,1:3] = θ_err 
    x_errs[i,7:18] = encs - x_eq[8:19]
    x_errs[i,19:21] = ω 
    x_errs[i,22:24] = v 
    x_errs[i,25:36] = joint_vels
    us[i,:] = -K * x_errs[i,:] * 0+ u_eq
    
    xs[i+1, :], λs[i+1,:] = semi_implicit_euler(A1, xs[i,:], us[i,:], ϕ_cons, foot_indices, h)
end

In [5]:
vis = Visualizer() 
cur_path = pwd()
cd(joinpath(@__DIR__,"..","src", "a1", "urdf"))
mvis = MechanismVisualizer(A1mech, URDFVisuals(urdfpath), vis)
cd(cur_path)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/chiyen/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


In [41]:
q_anim = [xs[i,1:19] for i in 1:length(times)-1]
animation = Animation(mvis, times[1:50:end-1], q_anim[1:50:end])
setanimation!(mvis, animation);

In [7]:
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x77, 0x6f, 0x72, 0x6c, 0x64, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("trunk" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x62, 0x61, 0x73, 0x65, 0x2f, 0x74, 0x72, 0x75, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_floating_base" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x61, 0x74, 0x69, 0x6e, 0x67, 0x5f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "RL_hip" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x75, 0x6e, 0x6b, 0x2f, 0x52, 0x4c, 0x5f, 0x68, 0x69, 0x70], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("RL_thigh" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x70, 0x2f, 0x52, 0x4c, 0x5f, 0x74, 0x68, 0x69, 0x67, 0x68], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_RL_thigh_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x69, 0x67, 0x68, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_17" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x37], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x37], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "RL_calf" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x68, 0x2f, 0x52, 0x4c, 0x5f, 0x63, 0x61, 0x6c, 0x66], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("RL_foot" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6c, 0x66, 0x2f, 0x52, 0x4c, 0x5f, 0x66, 0x6f, 0x6f, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_RL_foot_fixed" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x6f, 0x6f, 0x74, 0x5f, 0x66, 0x69, 0x78, 0x65, 0x64], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_19" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x39], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72,